In [15]:
import csv
import cv2
import numpy as np
import pickle

lines = []
with open("data/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
for line in lines:
    source_path=line[0]
    filename = source_path.split("/")[-1]
    image = cv2.imread("data/IMG/"+filename)
#     print("Reading {}".format("data/IMG"+filename))
#     print("image shape = {}".format(image.shape))
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

X_train = np.array(images)
y_train = np.array(measurements)

print("Shape of X_train = {}".format(X_train.shape))
print("Shape of y_train = {}".format(y_train.shape))

f = open("data.pkl", "wb")
pickle.dump({"x": X_train, "y": y_train}, f)


Shape of X_train = (1132, 160, 320, 3)
Shape of y_train = (1132,)


In [16]:

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D

data = pickle.load(open("data.pkl", "rb"))

X_train = data["x"]
y_train = data["y"]

# x_max = np.max(X_train)
# x_min = np.min(X_train)
# X_train = -0.5 + (X_train-x_min)/(x_max-x_min)

print("Shape of X_train = {}".format(X_train.shape))
print("Shape of y_train = {}".format(y_train.shape))

model = Sequential()
model.add(Lambda( lambda x : x/255 - 0.5, input_shape=(160, 320, 3)))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(Convolution2D(16, 3, 3, border_mode='same'))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10)

model.save('model.h5')


Shape of X_train = (1132, 160, 320, 3)
Shape of y_train = (1132,)
Train on 905 samples, validate on 227 samples
Epoch 1/10
905/905 [==============================] - 26s - loss: 8.3005 - val_loss: 0.3244